In [1]:
from admm_lasso import admm_lasso, plot
import casadi as ca
import numpy as np
from scipy import sparse

ref: https://www.researchgate.net/publication/299465495_An_Augmented_Lagrangian_Based_Algorithm_for_Distributed_NonConvex_Optimization

In [2]:
def create_prox_func(Nx,f_func):
    '''
    Not used here.
    '''
    x = ca.SX.sym("x",Nx)
    v = ca.SX.sym("v",Nx)
    lambda_ = ca.SX.sym("lambda",1)
    
    f = f_func(x)
    f_prox = f + 1/2 * lambda_ * ca.norm_2(x - v) ** 2
    f_prox_func = ca.Function("f_prox_func", [lambda_, x, v], [f_prox])

    p = ca.vertcat(v,lambda_)
    
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'f':f_prox, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

$\min _{y_{i}} f_{i}\left(y_{i}\right)+\lambda_{i}^{\top} A_{i} y_{i}+\frac{p}{2}\left\|A_{i}\left(y_{i}-x_{i}\right)\right\|_{2}^{2} \quad$ s.t. $\quad h_{i}\left(y_{i}\right) \leq 0$

In [3]:
def create_subproblem(fi_func, Ai, rho):
    N_lambda_i, N_yi = np.shape(Ai)
    yi = ca.SX.sym("yi",N_yi)
    xi = ca.SX.sym("xi",N_yi)
    lambda_i = ca.SX.sym("lambda_i",N_lambda_i)
    
    fi = fi_func(yi) + lambda_i.T @ Ai @ yi + rho/2 * ca.norm_2(Ai @ (yi - xi))**2 
    p = ca.vertcat(lambda_i, xi)
    g = yi
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':yi, 'g':g, 'f':fi, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [4]:
def create_QP_problem(A_list,rho):
    N = len(A_list)
    x_plus_list = []
    y_i_list = []
    lambda_plus_list = []
    obj = 0
    g = 0
    for i in range(N):
        Ai = A_list[i]
        
        N_lambda_i, N_yi = np.shape(Ai)
        yi = ca.SX.sym("yi",N_yi)
        x_plus = ca.SX.sym("xi",N_yi)
        lambda_plus = ca.SX.sym("lambda_i",N_lambda_i)
        
        x_plus_list += [x_plus]
        y_i_list += [yi] 
        lambda_plus_list += [lambda_plus]
    
        obj += rho/2 * ca.norm_2(Ai @ (yi - x_plus))**2  - lambda_plus.T @ Ai @ yi
        g += Ai @ x_plus
    x = ca.vertcat(*x_plus_list)
    p = ca.vertcat(*(lambda_plus_list + y_i_list))
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'g':g, 'f':obj, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver    

### Example 1
$\min _{x} x_{1} \cdot x_{2} \quad$ s.t. $\quad x_{1}-x_{2}=0$

analytical solution of $y$ is: $y=\left(\begin{array}{c}-2 \\ 2\end{array}\right) \lambda$

Numerical problem of ADMM with divergent $\lambda^{+} = - 2 \lambda$

In [5]:
eps = 1e-5
rho = 0.75
N_itermax = 50
A_list = []
fi_func_list = []

A = ca.DM([[1,-1]])
A_list += [A]
N = len(A_list)
b = ca.DM([0])

Nx = 2
x = ca.SX.sym("x",Nx)
fi_func = ca.Function("fi_func", [x], [x[0] * x[1]])
fi_func_list += [fi_func]

subsolver_list = []
# Define subproblem solvers
for i in range(N):
    Ai = A_list[i]
    fi_func = fi_func_list[i]
    subsolver_list += [create_subproblem(fi_func, Ai, rho)]
QP_solver = create_QP_problem(A_list, rho)
# Initial guess
xi_list = []
yi_list = []
lambda_i_list = []
lbx_list = []
ubx_list = []
for i in range(N):
    Ai = A_list[i]
    N_lambda_i, N_xi = np.shape(Ai)
    
    xi = np.random.randn(N_xi,1).flatten().tolist()
    lambda_i = np.random.randn(N_lambda_i,1).flatten().tolist()
    
    xi_list += [xi]
    lambda_i_list += [lambda_i]
    lbx_list += [[-ca.inf] * N_xi]
    ubx_list += [[ca.inf] * N_xi]
    yi_list += [[0] * N_xi]

nl_sub = {}

nl_qp = {}
nl_qp['lbg'] = b
nl_qp['ubg'] = b
nl_qp['lbx'] = sum(lbx_list,[])
nl_qp['ubx'] = sum(ubx_list,[])

# Track solution
yi_sol_list = []
x_plus_sol_list = []
lambda_i_sol_list = []
yi_sol_list += yi_list
x_plus_sol_list += [sum(xi_list,[])]
lambda_i_sol_list = [sum(lambda_i_list,[])]
for i in range(N_itermax):
    sum_Ay = 0
    for j in range(N):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
        nl_sub['p'] = lambda_i_list[j] + xi_list[j]
        
        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
        yi_sol_list += [yi_list[j]]
        
        sum_Ay += Ai @ yi_sol['x']
        lambda_i_list[j] = (ca.DM(lambda_i_list[j]) + rho * Ai @ (ca.DM(yi_list[j]) - ca.DM(xi_list[j]))).full().flatten().tolist()
        lambda_i_sol_list += [lambda_i_list[j]]
    if ca.norm_1(sum_Ay - b) <= eps:
        break
    
    nl_qp['x0'] = sum(xi_list,[])    #  2D list to 1D
    nl_qp['p'] = sum(lambda_i_list,[]) + sum(xi_list,[])
    xi_sol = solver_subproblem(**nl_qp)
    # Update x_plus
    pos = 0
    for j in range(N):
        
        xi_plus_list = xi_sol['x'].full().flatten().tolist()
        list_len = len(xi_list[j])
        xi_list[j] = xi_plus_list[pos:pos+list_len]
        pos += list_len
    
    x_plus_sol_list += [xi_plus_list]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [6]:
yi_sol_list

[[0, 0],
 [-0.06550244771683644, 0.06550244771683644],
 [0.13100489543367289, -0.13100489543367289],
 [-0.26200979086734577, 0.26200979086734577],
 [0.5240195817346915, -0.5240195817346915],
 [-1.048039163469383, 1.048039163469383],
 [2.096078326938766, -2.096078326938766],
 [-4.192156653877532, 4.192156653877532],
 [8.384313307755065, -8.384313307755065],
 [-16.76862661551013, 16.76862661551013],
 [33.53725323102026, -33.53725323102026],
 [-67.07450646204052, 67.07450646204052],
 [134.14901292408106, -134.14901292408106],
 [-268.2980258481622, 268.2980258481622],
 [536.5960516963245, -536.5960516963245],
 [-1073.1921033926487, 1073.1921033926487],
 [2146.384206785298, -2146.384206785298],
 [-4292.768413570595, 4292.768413570595],
 [8585.536827141192, -8585.536827141192],
 [-17171.07365428238, 17171.07365428238],
 [34342.14730856477, -34342.14730856477],
 [-68684.29461712952, 68684.29461712952],
 [137368.58923425907, -137368.58923425907],
 [-274737.1784685181, 274737.1784685181],
 [549

In [7]:
x_plus_sol_list

[[-0.44916723172205225, 0.4529304561834144],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0]]

In [8]:
lambda_i_sol_list

[[-0.6438220420706817],
 [-0.06550244771683644],
 [0.13100489543367289],
 [-0.26200979086734577],
 [0.5240195817346915],
 [-1.048039163469383],
 [2.096078326938766],
 [-4.192156653877532],
 [8.384313307755065],
 [-16.76862661551013],
 [33.53725323102026],
 [-67.07450646204052],
 [134.1490129240811],
 [-268.2980258481622],
 [536.5960516963245],
 [-1073.1921033926487],
 [2146.384206785298],
 [-4292.768413570595],
 [8585.536827141192],
 [-17171.07365428238],
 [34342.14730856477],
 [-68684.29461712952],
 [137368.58923425907],
 [-274737.1784685181],
 [549474.3569370363],
 [-1098948.7138740723],
 [2197897.427748145],
 [-4395794.855496289],
 [8791589.71099258],
 [-17583179.421985157],
 [35166358.84397032],
 [-70332717.68794063],
 [140665435.37588128],
 [-281330870.7517625],
 [562661741.5035251],
 [-1125323483.00705],
 [2250646966.0141006],
 [-4501293932.0282],
 [9002587864.056402],
 [-18005175728.1128],
 [36010351456.22556],
 [-72020702912.45105],
 [144041405824.90204],
 [-288082811649.8039],